# Actividad - Estadística básica

* **Nombre: Daniel Queijeiro Albo**
* **Matrícula: A01710441**

**Entregar:** Archivo PDF de la actividad, así como el archivo .ipynb en tu repositorio.
**Nota:** Recuerda habrá una penalización de **50** puntos si la actividad fue entregada fuera de la fecha límite.

Carga el conjunto de datos `insurance.csv` (se encuentra en el repositorio de la clase) y realiza un análisis estadístico de las variables.

In [3]:
# Carga las librerías necesarias.
import pandas as pd
import numpy as np
import random

In [4]:
# Carga el conjunto de datos al ambiente de Google Colab y muestra los primeros
# 6 renglones.
!curl -O https://raw.githubusercontent.com/Manchas2k4/tc1002S/main/datasets/insurance.csv

df = pd.read_csv('insurance.csv')
df.head(6)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54289  100 54289    0     0   946k      0 --:--:-- --:--:-- --:--:--  946k


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160


El conjunto de datos contiene información demográfica sobre los asegurados en una compañía de seguros:
* **age**: Edad del asegurado principal
* **sex**: Género del asegurado. female o male
* **bmi**: Índice de masa corporal
* **children**: Número de hijos que estan cubiertos con la poliza. 
* **smoke**: ¿El beneficiario fuma? (*yes*/*no*)
* **region**: ¿Dónde vive el beneficiario? Estos datos son de Estados Unidos. Regiones disponibles: northeast, southeast, southwest, northwest
* **charges**: Costo del seguro.

In [5]:
# Crea una tabla resumen con los estadísticas generales de las variables
# numéricas.
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [6]:
# ¿Cómo se correlacionan las varaibles numéricas entre sí?
df.corr()

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [12]:
# Determina si existe o no una correlación entre el índice de masa corporal 
# (bmi) y el costo del seguro.
selected = df[['age', 'bmi', 'children', 'charges']]
print('Correlación pearson', selected['bmi'].corr(selected['charges'], method='pearson'))
print('Correlación spearman', selected['bmi'].corr(selected['charges'], method='spearman'))
print('Correlación kendall', selected['bmi'].corr(selected['charges'], method='kendall'))

Correlación pearson 0.19834096883362895
Correlación spearman 0.11939590358331145
Correlación kendall 0.08252397079981415


In [13]:
# ¿Cuántas personas aseguradas son hombre y cuántas son mujeres?
ndf = df.dropna(subset=['charges'])
h = ndf.groupby('sex').get_group('male')
m = ndf.groupby('sex').get_group('female')
print('Hay', len(h),'hombres asegurados y ', len(m), 'mujeres aseguradas.')

Hay 676 hombres asegurados y  662 mujeres aseguradas.


In [15]:
# ¿Cuántos hombres y mujeres asegurados viven en cada región?
group = ndf.groupby('region')
count = group['sex'].value_counts()
resultado = pd.DataFrame(count)
resultado.columns = ['Cantidad']
print(resultado)

                  Cantidad
region    sex             
northeast male         163
          female       161
northwest female       164
          male         161
southeast male         189
          female       175
southwest male         163
          female       162


In [16]:
# En promedio, ¿quién paga más de cuota de seguro? ¿Los fumadores o los no 
# fumadores? Muéstralo con los datos.
promedio = df.groupby('smoker')['charges'].mean()
print("La cuota promedio de fumadores es de ", promedio['yes'])
print("La cuota promedio de no fumadores es de ", promedio['no'])

La cuota promedio de fumadores es de  32050.23183153285
La cuota promedio de no fumadores es de  8434.268297856202


In [17]:
# ¿Cuáles son las cuotas mínimas y máximas que las personan pagan dependiendo
# del género y del número de hijos?
df.groupby(['sex','children']).agg(['min','max'])[['charges']]

charges             
                        min          max
sex    children                         
female 0         1607.51010  63770.42801
       1         2201.09710  58571.07448
       2         2801.25880  47305.30500
       3         4234.92700  46661.44240
       4         4561.18850  36580.28216
       5         4687.79700  19023.26000
male   0         1121.87390  62592.87309
       1         1711.02680  51194.55914
       2         2304.00220  49577.66240
       3         3443.06400  60021.39897
       4         4504.66240  40182.24600
       5         4915.05985  14478.33015

In [18]:
# ¿Cuál es el índice de masa corporal promedio para hombre y mujeres dependiendo
# región en la que viven y si son fumadores? ¿Impacta eso en la tarifa del 
# seguro?
promedio_bmi = ndf.groupby(['sex', 'region', 'smoker'])['bmi', 'charges'].mean()
print("Promedio de bmi y tarifa por genero, región y fumadores")
print(promedio_bmi)

Promedio de bmi y tarifa por genero, región y fumadores
                               bmi       charges
sex    region    smoker                         
female northeast no      29.777462   9640.426984
                 yes     27.261724  28032.046398
       northwest no      29.488704   8786.998679
                 yes     28.296897  29670.824946
       southeast no      32.780000   8440.205552
                 yes     32.251389  33034.820716
       southwest no      30.050355   8234.091260
                 yes     30.128571  31687.988430
male   northeast no      28.861760   8664.042222
                 yes     29.560000  30926.252583
       northwest no      28.930379   8320.689321
                 yes     29.983966  30713.181419
       southeast no      34.129552   7609.003587
                 yes     33.650000  36029.839367
       southwest no      31.019841   7778.905534
                 yes     31.502703  32598.862854


<ipython-input-18-0a5db88234c3>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  promedio_bmi = ndf.groupby(['sex', 'region', 'smoker'])['bmi', 'charges'].mean()
